In [5]:
import pickle

train_data = pickle.load(open('../data/diginetica/raw/train.txt', 'rb'))

In [13]:
len(train_data)

2

In [14]:
type(train_data)

tuple

In [15]:
train_data[1]

[2,
 5,
 4,
 7,
 9,
 15,
 14,
 13,
 12,
 12,
 11,
 18,
 18,
 17,
 20,
 19,
 21,
 22,
 28,
 24,
 27,
 26,
 25,
 24,
 31,
 30,
 36,
 35,
 34,
 33,
 4,
 5,
 39,
 38,
 41,
 43,
 45,
 46,
 47,
 50,
 49,
 52,
 56,
 22,
 56,
 55,
 53,
 54,
 59,
 58,
 68,
 68,
 67,
 66,
 62,
 65,
 64,
 62,
 63,
 62,
 61,
 61,
 69,
 75,
 74,
 73,
 72,
 71,
 77,
 79,
 78,
 9,
 82,
 84,
 86,
 88,
 11,
 11,
 88,
 87,
 96,
 95,
 94,
 93,
 92,
 91,
 90,
 89,
 99,
 98,
 101,
 110,
 109,
 108,
 107,
 106,
 105,
 104,
 103,
 53,
 22,
 112,
 102,
 103,
 103,
 115,
 114,
 117,
 53,
 9,
 116,
 126,
 125,
 124,
 123,
 122,
 121,
 120,
 119,
 119,
 118,
 118,
 119,
 130,
 129,
 128,
 132,
 131,
 134,
 138,
 136,
 137,
 135,
 144,
 143,
 139,
 142,
 141,
 140,
 139,
 138,
 137,
 135,
 136,
 142,
 139,
 150,
 145,
 149,
 119,
 148,
 147,
 146,
 152,
 154,
 19,
 156,
 155,
 159,
 158,
 161,
 163,
 164,
 165,
 168,
 167,
 172,
 171,
 170,
 3,
 173,
 173,
 175,
 174,
 176,
 180,
 179,
 145,
 178,
 148,
 182,
 184,
 190,
 189,
 1

In [16]:
train_data[0]

[[1],
 [3, 4],
 [3],
 [6],
 [8],
 [10, 11, 12, 12, 13, 14],
 [10, 11, 12, 12, 13],
 [10, 11, 12, 12],
 [10, 11, 12],
 [10, 11],
 [10],
 [16, 17, 18],
 [16, 17],
 [16],
 [19, 19],
 [19],
 [21],
 [8],
 [23, 24, 25, 26, 27, 24],
 [23, 24, 25, 26, 27],
 [23, 24, 25, 26],
 [23, 24, 25],
 [23, 24],
 [23],
 [29, 30],
 [29],
 [32, 5, 4, 33, 34, 35],
 [32, 5, 4, 33, 34],
 [32, 5, 4, 33],
 [32, 5, 4],
 [32, 5],
 [32],
 [37, 38],
 [37],
 [40],
 [42],
 [44],
 [46, 47],
 [46],
 [48, 49],
 [48],
 [51],
 [53, 54, 53, 55, 56, 22],
 [53, 54, 53, 55, 56],
 [53, 54, 53, 55],
 [53, 54, 53],
 [53, 54],
 [53],
 [57, 58],
 [57],
 [60, 61, 61, 62, 63, 62, 64, 65, 62, 66, 67, 68],
 [60, 61, 61, 62, 63, 62, 64, 65, 62, 66, 67],
 [60, 61, 61, 62, 63, 62, 64, 65, 62, 66],
 [60, 61, 61, 62, 63, 62, 64, 65, 62],
 [60, 61, 61, 62, 63, 62, 64, 65],
 [60, 61, 61, 62, 63, 62, 64],
 [60, 61, 61, 62, 63, 62],
 [60, 61, 61, 62, 63],
 [60, 61, 61, 62],
 [60, 61, 61],
 [60, 61],
 [60],
 [69],
 [70, 71, 72, 73, 74],
 [70, 71

train_data[0] 是序列，train_data[1]是label

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, GatedGraphConv


class Embedding2Score(nn.Module):
    def __init__(self, hidden_size):
        super(Embedding2Score, self).__init__()
        self.hidden_size = hidden_size
        self.W_1 = nn.Linear(self.hidden_size, self.hidden_size)
        self.W_2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.q = nn.Linear(self.hidden_size, 1)
        self.W_3 = nn.Linear(2 * self.hidden_size, self.hidden_size)

    def forward(self, session_embedding, all_item_embedding, batch):
        sections = torch.bincount(batch)
        v_i = torch.split(session_embedding, tuple(sections.cpu().numpy()))  # split whole x back into graphs G_i
        v_n_repeat = tuple(nodes[-1].view(1, -1).repeat(nodes.shape[0], 1) for nodes in
                           v_i)  # repeat |V|_i times for the last node embedding

        # Eq(6)
        alpha = self.q(torch.sigmoid(self.W_1(torch.cat(v_n_repeat, dim=0)) + self.W_2(session_embedding)))  # |V|_i * 1
        s_g_whole = alpha * session_embedding  # |V|_i * hidden_size
        s_g_split = torch.split(s_g_whole, tuple(sections.cpu().numpy()))  # split whole s_g into graphs G_i
        s_g = tuple(torch.sum(embeddings, dim=0).view(1, -1) for embeddings in s_g_split)

        # Eq(7)
        v_n = tuple(nodes[-1].view(1, -1) for nodes in v_i)
        s_h = self.W_3(torch.cat((torch.cat(v_n, dim=0), torch.cat(s_g, dim=0)), dim=1))

        # Eq(8)
        z_i_hat = torch.mm(s_h, all_item_embedding.weight.transpose(1, 0))

        return z_i_hat


class GNNModel(nn.Module):
    """
    Args:
        hidden_size: the number of units in a hidden layer.
        n_node: the number of items in the whole item set for embedding layer.
    """

    def __init__(self, hidden_size, n_node):
        super(GNNModel, self).__init__()
        self.hidden_size, self.n_node = hidden_size, n_node
        self.embedding = nn.Embedding(self.n_node, self.hidden_size)
        self.gated = GatedGraphConv(self.hidden_size, num_layers=1)
        self.e2s = Embedding2Score(self.hidden_size)
        self.loss_function = nn.CrossEntropyLoss()
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def forward(self, data):
        x, edge_index, batch = data.x - 1, data.edge_index, data.batch

        embedding = self.embedding(x).squeeze()
        hidden = self.gated(embedding, edge_index)
        hidden2 = F.relu(hidden)

        return self.e2s(hidden2, self.embedding, batch)